# 🏛️ Semantic Objects - Basic Tutorial

Welcome to the **Semantic Objects** library! This tutorial will walk you through the core functionality step by step.

## What You'll Learn

1. **Creating semantic objects** - Define spaces, windows, and properties
2. **Working with properties** - Handle values, units, and quantity kinds
3. **Template generation** - Export BuildingMOTIF templates
4. **SHACL validation** - Generate validation rules
5. **Model building** - Create RDF models with BMotifSession

Let's get started! 🚀

## Setup and Imports

In [ ]:
# Core imports
from semantic_objects.s223 import *
from semantic_objects.core import export_templates
from semantic_objects.build_model import BMotifSession
from pprint import pprint
import pandas as pd

print("✅ Imports successful!")

## 1. 🔍 Exploring Entities and Properties

In this library, **Entities** (like a Space or Window) represent physical or logical objects, while **Properties** define the attributes associated with them.

### Inspecting an Entity

Let's look at what fields are available for a `Space`:

In [ ]:
# View the fields associated with a Space
print("Space fields:")
pprint(Space.__dataclass_fields__)

print("\n" + "="*50)
print("Window fields:")
pprint(Window.__dataclass_fields__)

### Diving into Property Metadata

Properties are defined with specific values, units, and **Quantity Kinds**:

In [ ]:
# Inspect the metadata of the Area property
print(f"Property Class: {Area}")
print("\nArea property fields:")
pprint(Area.__dataclass_fields__)

# Look at the quantity kind
print(f"\nArea quantity kind: {Area.qk}")
print(f"Quantity kind IRI: {Area.qk._get_iri()}")

## 2. 🏗️ Creating Semantic Objects

Let's create some semantic objects and see how they work:

In [ ]:
# Create a simple space with area
office = Space(area=150.0)  # Area in default units (ft²)

print(f"Created space with area: {office.area.value} {office.area.unit}")
print(f"Space name: {office._name}")
print(f"Area quantity kind: {office.area.qk}")

In [ ]:
# Create a window with multiple properties
south_window = Window(
    area=25.0,      # ft²
    azimuth=180.0,  # degrees (south-facing)
    tilt=90.0       # degrees (vertical)
)

print(f"Window area: {south_window.area.value} {south_window.area.unit}")
print(f"Window azimuth: {south_window.azimuth.value}° (south-facing)")
print(f"Window tilt: {south_window.tilt.value}° (vertical)")
print(f"Window name: {south_window._name}")

### Working with Different Units

In [ ]:
# Import specific units
from semantic_objects.qudt.units import M2, FT2

# Create area with explicit metric units
metric_area = Area(50.0, unit=M2)
imperial_area = Area(50.0, unit=FT2)

print(f"Metric area: {metric_area.value} {metric_area.unit}")
print(f"Imperial area: {imperial_area.value} {imperial_area.unit}")

# Create space with metric area
metric_space = Space(area=metric_area)
print(f"\nMetric space area: {metric_space.area.value} {metric_space.area.unit}")

## 3. 📝 Template Generation

A core feature is exporting these objects into **YAML templates** for use by `buildingmotif`:

In [ ]:
# Export templates to a directory
export_templates(Space, 'templates')

print("✅ Templates exported to 'templates/' directory")
print("Generated files:")
import os
for file in os.listdir('templates'):
    print(f"  - {file}")

### Inspecting Generated Templates

Let's see what the generated templates look like:

In [ ]:
# Get related classes for Space
related_classes = get_related_classes(Space)
predicates, entities, values = related_classes

print(f"Found {len(predicates)} predicates, {len(entities)} entities, {len(values)} values")
print("\nEntities:")
for entity in entities:
    print(f"  - {entity.__name__}")

print("\nValues (Properties):")
for value in values:
    print(f"  - {value.__name__}")

In [ ]:
# Look at a generated YAML template
print("Example generated YAML template for Space:")
print(Space.generate_yaml_template())

## 4. ⚖️ Validation and SHACL Generation

The library can generate **RDF class definitions** and **SHACL shapes** to ensure your models adhere to ontology constraints:

In [ ]:
# Generate RDF definition including the parent hierarchy (RDFS inheritance)
print("=== Full Hierarchy ===")
full_hierarchy = Space.generate_rdf_class_definition(include_hierarchy=True)
print(full_hierarchy[:500] + "..." if len(full_hierarchy) > 500 else full_hierarchy)

In [ ]:
# Generate just the local class constraints
print("=== Local Constraints Only ===")
local_constraints = Space.generate_rdf_class_definition(include_hierarchy=False)
print(local_constraints)

### SPARQL Query Generation

The library automatically generates SPARQL queries from class definitions:

In [ ]:
# Generate SPARQL query for Space class
space_query = Space.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Space:")
print(space_query)

In [ ]:
# Generate query for Window (more complex with multiple properties)
window_query = Window.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Window:")
print(window_query)

## 5. 🏗️ Building Models with BMotifSession

The `BMotifSession` class integrates with BuildingMOTIF to create RDF models:

In [ ]:
# Create a BMotifSession
session = BMotifSession(ns='tutorial')

# Load class templates
session.load_class_templates(Space)

print("✅ BMotifSession created and templates loaded")
print(f"Available templates: {list(session.templates.keys())}")

In [ ]:
# Look at template parameters
space_template = session.templates['Space']
print("Space template parameters:")
print(space_template.inline_dependencies().all_parameters)

In [ ]:
# Create a space and evaluate it
conference_room = Space(area=200.0)
conference_room._name = "ConferenceRoom_101"

print("Space field values:")
pprint(conference_room.get_field_values(recursive=True))

In [ ]:
# Evaluate the space (generate RDF)
session.evaluate(conference_room)

print("✅ Space evaluated and added to model")
print("\nGenerated RDF (first 1000 characters):")
rdf_output = session.graph.serialize(format='turtle')
print(rdf_output[:1000] + "..." if len(rdf_output) > 1000 else rdf_output)

## 6. 🔄 Working with Multiple Objects

Let's create a more complex example with multiple spaces and windows:

In [ ]:
# Create multiple spaces
spaces = [
    Space(area=150.0),  # Office 1
    Space(area=200.0),  # Conference room
    Space(area=100.0),  # Office 2
]

# Set names
space_names = ["Office_101", "ConferenceRoom_201", "Office_102"]
for space, name in zip(spaces, space_names):
    space._name = name

print("Created spaces:")
for space in spaces:
    print(f"  - {space._name}: {space.area.value} {space.area.unit}")

In [ ]:
# Create windows for the spaces
windows = [
    Window(area=20.0, azimuth=90.0, tilt=90.0),   # East-facing
    Window(area=30.0, azimuth=180.0, tilt=90.0),  # South-facing
    Window(area=15.0, azimuth=270.0, tilt=90.0),  # West-facing
]

window_names = ["Window_E101", "Window_S201", "Window_W102"]
for window, name in zip(windows, window_names):
    window._name = name

print("Created windows:")
for window in windows:
    direction = {90.0: "East", 180.0: "South", 270.0: "West"}[window.azimuth.value]
    print(f"  - {window._name}: {window.area.value} ft², {direction}-facing")

In [ ]:
# Create a new session and evaluate all objects
building_session = BMotifSession(ns='building')

# Load templates for both Space and Window
building_session.load_class_templates(Space)
building_session.load_class_templates(Window)

print(f"Available templates: {list(building_session.templates.keys())}")

In [ ]:
# Evaluate all spaces and windows
for space in spaces:
    building_session.evaluate(space)
    
for window in windows:
    building_session.evaluate(window)

print("✅ All objects evaluated")

# Count triples in the graph
triple_count = len(building_session.graph)
print(f"Generated {triple_count} RDF triples")

In [ ]:
# Show a sample of the generated RDF
print("Sample of generated RDF:")
rdf_sample = building_session.graph.serialize(format='turtle')
lines = rdf_sample.split('\n')
print('\n'.join(lines[:30]))  # Show first 30 lines
if len(lines) > 30:
    print(f"... and {len(lines) - 30} more lines")

## 7. 📊 Summary and Next Steps

Congratulations! You've learned the basics of Semantic Objects:

✅ **Created semantic objects** (Space, Window) with properties (Area, Azimuth, Tilt)  
✅ **Worked with units** and quantity kinds  
✅ **Generated templates** for BuildingMOTIF  
✅ **Created SHACL shapes** for validation  
✅ **Built RDF models** using BMotifSession  
✅ **Generated SPARQL queries** automatically  

### What's Next?

1. **Model Loading Tutorial**: Learn to load RDF data into Python objects
2. **Advanced Examples**: Complex relationships and custom entities
3. **Template Generation**: Deep dive into BuildingMOTIF integration
4. **Custom Entities**: Create your own semantic object types

### Key Takeaways

- **Pythonic Interface**: Define ontology concepts as simple dataclasses
- **Automatic Generation**: SPARQL queries, SHACL shapes, and templates generated automatically
- **Type Safety**: Full type hints and validation
- **Flexible Units**: Support for metric and imperial units
- **BuildingMOTIF Integration**: Seamless template generation and model building

Happy modeling! 🎉